## Project name

### 1.运行前准备

#### 1.1依赖模块
```bash
pip3 install numpy # 数学库
pip3 install matplotlib # 绘图库
pip3 install tqdm # 进度条模块
pip3 install ipywidgets # 进度条相关依赖
pip3 install pillow # 图像处理
pip3 install opencv-python # 视觉库
```

#### 1.2其它

以下代码中使用了tqdm模块来绘制进度条，并且需要安装ipywidgets模块来在jupyter中显示。

```bash
pip3 install ipywidgets
```

如果有相关报错请安装或更新这两个模块或者注释掉相关代码（对应代码均有注释说明）
```python
# 绘制进度条
p = tqdm(total=idate_count)
# 更新进度条
p.update(1)
# 结束进度条
p.close()
```

### 2.导入相关模块

In [ ]:
import os
import csv
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
from PIL import Image, ImageDraw, ImageFont

### 3.数据处理

先用excel将xlsx文件导出为csv文件，方便处理

#### 3.1读取文件

In [ ]:
# 读取csv文件
with open("国家地铁里程历程.csv", 'r',encoding='UTF-8-SIG') as f:
    csv_reader = list(csv.reader(f))
    csv_head = csv_reader[0]
    csv_body = csv_reader[1:]

#### 3.2数据预处理

In [ ]:
# csv_body 每一列最前面的正整数之前全部设为 0

country_count = len(csv_head)-1 # 国家数量

previous_status = [0 for i in range(country_count)]
for i in range(len(csv_body)):
    for j in range(country_count):
        if previous_status == [1 for i in range(country_count)]:
            break

        if csv_body[i][j+1] == '' and previous_status[j] == 0:
            csv_body[i][j+1] = '0'
        else:
            previous_status[j] = 1


In [ ]:
# 生成起始数据
start_data = csv_body[0][1:]
for i in csv_body[0:2]:
    for j in range(country_count):
        if i[j + 1] != '':
            start_data[j] = i[j + 1]


In [ ]:
# 向列表中插入需要插值的日期

csv_body_idate = []
for i in csv_body[2:]:
    for j in range(1, 13):
        if j == 12:
            temp = [[i[0] + "." + str(j)] + i[1:]]
        else:
            temp = [[i[0] + "." + str(j)] + ['' for i in range(country_count)]]
        csv_body_idate += temp

csv_body_idate[0] = ['1900.1'] + start_data


#### 3.3对数据进行插值

In [ ]:
# 对列表进行线性插值

idate_count = len(csv_body_idate)

# 转置矩阵，方便插值
interp_arr = np.transpose(np.array(csv_body_idate))

x = [i for i in range(1,idate_count+1)]

out_arr = np.array([interp_arr[0]])

for i in range(country_count):
    xp = []
    yp = []
    for j in range(idate_count):
        if interp_arr[i + 1][j] != '':
            xp.append(float(x[j]))
            yp.append(float(interp_arr[i + 1][j]))
    y = np.interp(x, xp, yp)
    out_arr = np.append(out_arr, [y], axis=0)

# 数据只保留两位小数
out_arr = [out_arr[0].tolist()] + list(map(lambda x:list(map(lambda y:round(float(y),2),x)),out_arr[1:]))

# 转置矩阵
out_arr = np.transpose(np.array(out_arr))

# 输出到列表
csv_body_out = out_arr.tolist()


#### 3.4输出文件

In [ ]:
# 保存列表

with open("output.csv", 'w',encoding='UTF-8-SIG',newline='') as f:
   writer = csv.writer(f)
   writer.writerow(csv_head)
   writer.writerows(csv_body_out)

### 4.绘图

#### 4.1定义相关函数

In [ ]:
# 定义绘制设置函数
def draw_setting(**kwargs):

    kw={'x':1920,'y':1080,'set_dpi':300}
    for k,v in kwargs.items():
        if kw.get(k) != None:
            kw[k] = v

    plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
    plt.rcParams['axes.unicode_minus']=False #用来正常显示负号 #有中文出现的情况，需要u'内容'

    # # 去掉图例 legend=False
    # # 缩小柱间距 width=0.8
    fig, ax = plt.subplots(figsize=(kw['x']/kw['set_dpi'],kw['y']/kw['set_dpi']),dpi=kw['set_dpi'])

    # fig = plt.figure(figsize=(1920/set_dpi,1080/set_dpi),dpi=set_dpi)
    # ax = plt.gca()

    # 图的左边缘位置
    fig.subplots_adjust(left=0.1)

    # 逆序显示 y 轴
    ax.invert_yaxis()

    # 去除四周的边框 (spine.set_visible(False))
    [spine.set_visible(False) for spine in ax.spines.values()]

    # 去除 x 和 y 轴之间无用的刻度 tick
    # 去除 x 轴上无用的标签
    ax.tick_params(bottom=False, left=False, labelbottom=False)

    # 增加 y 轴标签的尺寸
    ax.tick_params(axis='y', labelsize='medium')

    return fig,ax


#### 4.2绘图相关设置

In [ ]:
# 设置dpi
set_dpi = 300

# 里程列表
mileage_list = list(
    map(lambda x: list(map(float, x)), [i[1:] for i in csv_body_out]))

# 日期列表
# date_list = [i[0] for i in csv_body_out]

# 国家列表
country_list = csv_head[1:]

# 国家颜色
color_dict = {
    '英国': '#FF8000',
    '巴西': '#FFFF00',
    '法国': '#80FF00',
    '德国': '#00FFFF',
    '西班牙': '#8000FF',
    '俄罗斯': '#FF00FF',
    '日本': '#FF7A7A',
    '印度': '#0000FF',
    '韩国': '#CA7AFF',
    '美国': '#0080FF',
    '中国': '#FF0000'
}

# 图片保存路径
image_path = './image/'

# 自动创建目录
if not os.path.exists(image_path):
    os.mkdir(image_path)

#### 4.3进行绘制


In [ ]:

# 绘制进度条
p = tqdm(total=idate_count)

# 循环绘制
for i in range(idate_count):

    plt.clf()

    fig, ax = draw_setting(set_dpi=set_dpi)

    vmax = max(mileage_list[i])

    z = zip(country_list, mileage_list[i])

    # 从大到小排列
    sort_z = sorted(z, key=lambda x: x[1], reverse=True)

    filter_z = []
    # 过滤里程为0的
    for j in sort_z:
        if j[1] != 0:
            filter_z.append(j)

    # 限制十个国家
    filter_z = filter_z[:10] if len(filter_z) > 10 else filter_z

    # 绘制
    for country, mileage in filter_z:

        color = color_dict[country]

        ax.barh(country, mileage, color=color)

        # 右侧实际的值展示
        ax.text(mileage + vmax * 0.02,
                country,
                f'{mileage:,}',
                fontsize='medium',
                va='center',
                color='C0')

    # 保存图片
    plt.savefig(image_path + str(i) + '.png', transparent=True, dpi=set_dpi)

    # 更新进度条
    p.update(1)

# 结束进度条
p.close()


### 5.图片处理

#### 5.1添加背景图片

In [ ]:

#加载背景图片
background_img = Image.open('./background.png')

# 绘制进度条
p = tqdm(total=idate_count)

for i in range(idate_count):

    # 加载柱状图
    chart_img = Image.open('./image/'+str(i)+'.png')

    #新建透明底图，mode使用RGBA，保留Alpha透明度，颜色为透明
    #Image.new(mode, size, color=0)，color可以用tuple表示，分别表示RGBA的值
    target_img = Image.new('RGBA', background_img.size, (0, 0, 0, 0))

    #将背景贴到目标图像
    target_img.paste(background_img,(0,0)) 
    
    #将条形图贴到目标图像并保留透明度
    target_img.paste(chart_img,(0,0),chart_img)#第一个参数表示需要粘贴的图像，中间的是坐标，最后是一个是mask图片，用于指定透明区域，将底图显示出来。

    # target.show()
    target_img.save('./image/'+str(i)+'.png')  # 保存图片

    # 更新进度条
    p.update(1)

# 结束进度条
p.close()

#### 5.2添加文字

In [ ]:
# 绘制进度条
p = tqdm(total=idate_count)

for i in range(idate_count):
    target_img = Image.open('./image/'+str(i)+'.png')

    draw = ImageDraw.Draw(target_img)

    # 绘制标题
    title_fontStyle = ImageFont.truetype("simhei.ttf", 80, encoding="utf-8")
    draw.text((620, 30), "全球各国地铁里程", "#262626", font=title_fontStyle)

    # 绘制年份
    year = str(1900+int(i/60)*5) # 每5年更新一次
    year_fontStyle = ImageFont.truetype("simhei.ttf", 100, encoding="utf-8")
    draw.text((1460, 800), year, "#bfbfbf", font=year_fontStyle)

    # 绘制班级姓名水印
    # 字体的格式 这里的simhei.ttf需要有这个字体
    mark_fontStyle = ImageFont.truetype("simhei.ttf", 20, encoding="utf-8")
    draw.text((5, 5), "20软件6班张某人", "#737373", font=mark_fontStyle)

    target_img.save('./image/'+str(i)+'.png')  # 保存图片

    # 更新进度条
    p.update(1)

# 结束进度条
p.close()

### 6.视频合成

以下使用opencv模块来合成视频，opencv合成出来的视频质量可能不高

如需合成更清晰的视频并加上背景音乐可以用ffmpeg，或者直接用视频处理软件来处理

In [ ]:
import cv2

# 绘制进度条
p = tqdm(total=idate_count)

# DIVX是实现 MPEG-4 标准的编码器
video=cv2.VideoWriter('./output.mp4',cv2.VideoWriter_fourcc(*'DIVX'),12,(1920,1080))  #定义保存视频目录名称及压缩格式，fps=10,像素为1920*1080
for i in range(idate_count):
    img=cv2.imread('./image/'+str(i)+'.png',1)  #读取图片
    # img=cv2.resize(img,(1920,1080)) #将图片转换为1920*1080
    video.write(img)   #写入视频
    # 更新进度条
    p.update(1)

video.release()

# 结束进度条
p.close()
